In [ ]:
import urllib
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
import pickle
from collections import Counter
from collections import namedtuple
import re
import xlwings as xw
import pandas as pd
import numpy as np

In [ ]:
path_to_names = "D:/"
path_to_patronymics = "D:/"

In [3]:
def distance(s, t):
    if s == t: return 0
    elif len(s) == 0: return len(t)
    elif len(t) == 0: return len(s)
    v0 = [None] * (len(t) + 1)
    v1 = [None] * (len(t) + 1)
    for i in range(len(v0)):
        v0[i] = i
    for i in range(len(s)):
        v1[0] = i + 1
        for j in range(len(t)):
            cost = 0 if s[i] == t[j] else 1
            v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
        for j in range(len(v0)):
            v0[j] = v1[j]
    return v1[len(t)]


In [ ]:
def checkSurnames(s):
    pattern = '\w*(ов|ова|ев|ева|ин|ина|ын|их|ых|ский|цкий|ая|ко)$'
    if (re.match(pattern,s)): return True
    else: return False

In [52]:
def Brute(tmp):
    res = []
    errs=[]
    for t in tmp:
        name = ""
        surname = ""
        patronymic = ""
        words = t.split(" ")
        for w in words:
            w = w.lower()
            w.strip()
            if (name == ""):
                if (w in all_names):
                    name = w

            if (patronymic == ""):
                if (w in all_patronymics):
                    patronymic = w

            if (name != w and patronymic != w and surname == ""):
                surname = w

        if (name == ""):
            name = "N:'"+w+"'_Not_Found"
            #errs.append("name '" + w + "' in " + t)
        if (patronymic == ""):
            patronymic = "P:'"+w+"'_Not_Found"
            errs.append("patronymic '" + w + "' in " + t)
        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname + " " + name + " " + patronymic)

    if len(errs)!=0:
        res.append("----Ошибки----")
        for i in errs:
            res.append(i)
    return res

In [ ]:
def NotBruteEnough(tmp):
    res = []

    for t in tmp:
        name = ""
        surname = ""
        patronymic = ""
        t = t.lower()
        words = t.split(" ")

        for w in words:
            w.strip()
            if (name == ""):
                if (w in all_names):
                    name = w

            if (patronymic == ""):
                if (w in all_patronymics):
                    patronymic = w

            if (name != w and patronymic != w and checkSurnames(w) and surname == ""):
                surname = w

        if (name in words): words.remove(name)
        if (patronymic in words): words.remove(patronymic)
        if (surname in words): words.remove(surname)


        if (name == "") & (len(words) > 0):
            foundWord = ""
            for w in words:

                minDist = len(w)
                for n in all_names:
                    dist = distance(w, n)
                    if dist < minDist:
                        minDist = dist
                        name = n.title() + " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (patronymic == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for p in all_patronymics:
                    dist = distance(w, p)
                    if dist < minDist:
                        minDist = dist
                        patronymic = p.title() + " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (surname == "")& (len(words) > 0):
            surname = words[0]

        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname + " " + name + " " + patronymic)

    return res

In [19]:
with open(path_to_names+"NamesFromGufo.pickle","rb") as f:
    tmpNames=pickle.load(f)
with open(path_to_patronymics+"PatronymicsFromGufo.pickle","rb") as f:
    tmpPatronymics=pickle.load(f)

In [20]:
all_names=[]
all_patronymics=[]

In [21]:
for i in tmpNames:
    all_names.append(i)
for i in tmpPatronymics:
    all_patronymics.append(i)

In [22]:
with open("D:/test_changed.pickle","rb") as f:
    tmp=pickle.load(f)

In [29]:
testing_res=NotBruteEnough(tmp)
for fio in testing_res:
    print(fio)

Стрелкова Глафира Савеловна (56% Стрелкова)
 Константин Николаевич
Субботина Оксана Глебовна (44% Субботина)
Бурова Милица (100% Милица) Евгеньевна
Сорокин Майор (60% Мэлор) Арсеньевич
Зуева Ева (60% Зуева) Лаврентьевна
Кузьмин Кузьма (71% Кузьмин) Матвеевич
Дементьева Юлия (100% Юлия) Леонидовна
Сорокин Афросин (57% Сорокин) Егорович
Попова Екатерина Михаиловна (100% Михаиловна)
 Варвара Руслановна
Турова Антонина Кондратовна (100% Кондратовна)
Улебовна Алевтина Игнатовна (88% Игнатова)
 Фрол Артёмович
Селезнёва Елена (56% Селезнёва) Якуновна
Нестеров Василий Нестерович (75% Нестеров)
Сидоров Виктор Михаилович (100% Михаилович)
 Олимпиада Андреевна
Овчинников Антонин Вероникович (40% Овчинников)
Путин Богдан (100% Богдан) Христофорович
Прохорова Дарья Авксентьевна (100% Авксентьевна)
Суворова Галина (100% Галина) Антониновна
Мэлс Макс (50% Мэлс) Арсеньевич
Самсонов Ростислав Антонович (100% Антонович)
Семёнова Семён (62% Семёнова) Варламовна
 Ангелина Святославовна
Егорова Егор (57% Е

In [ ]:
errs=0
for i in testing_res:
    if (i.find("%")!=-1):
        print (i)
        errs+=1
print("Всего не уверены в: " + str(errs))